In [ ]:
! pip install -q kaggle

In [ ]:
from google.colab import files

files.upload()

KeyboardInterrupt: 

In [ ]:
!echo '{"username":"amaithi","key":""}' > /root/.kaggle/kaggle.json

In [ ]:
! chmod 600 /root/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c jigsaw-toxic-comment-classification-challenge

jigsaw-toxic-comment-classification-challenge.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
! unzip jigsaw-toxic-comment-classification-challenge

Archive:  jigsaw-toxic-comment-classification-challenge.zip
replace sample_submission.csv.zip? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
! unzip /content/train.csv.zip

Archive:  /content/train.csv.zip
replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [ ]:

!pip install transformers

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('/content/train.csv', engine='python')

In [ ]:
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [ ]:
df['toxic'].value_counts()

toxic
0    144277
1     15294
Name: count, dtype: int64

In [ ]:
df = df[['comment_text', 'toxic']]
df = df[0:1000]
df.head()

,comment_text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import torch
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer, BertForSequenceClassification

In [ ]:
import os
from google.colab import userdata
os.environ['HF_TOKEN'] = userdata.get('HF_TOKEN')

In [ ]:
tokenizer = BertTokenizer.from_pretrained('google-bert/bert-base-uncased')

model = BertForSequenceClassification.from_pretrained('google-bert/bert-base-uncased', num_labels=2)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model = model.to('cuda')

In [ ]:
sample_data = ['I am eating', 'I am playing']

tokenizer(sample_data, padding=True, truncation=True, max_length=512)

{'input_ids': [[101, 1045, 2572, 5983, 102], [101, 1045, 2572, 2652, 102]], 'token_type_ids': [[0, 0, 0, 0, 0], [0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1], [1, 1, 1, 1, 1]]}

In [ ]:
X = list(df['comment_text'])
y = list(df['toxic'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, stratify = y)



In [ ]:
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=512)
X_test_tokenized = tokenizer(X_test, padding =True, truncation=True, max_length=512)

In [ ]:
X_train_tokenized.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [ ]:
print(X_train_tokenized['input_ids'][0])

[101, 20228, 2015, 2507, 1037, 7514, 1010, 2748, 2030, 2053, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [ ]:
len(X_train), len(X_test)

(800, 200)

In [ ]:
sample_data = ['I am eating', 'I am playing']

tokenizer(sample_data, padding=True, truncation=True, max_length=512)

{'input_ids': [[101, 1045, 2572, 5983, 102], [101, 1045, 2572, 2652, 102]], 'token_type_ids': [[0, 0, 0, 0, 0], [0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1], [1, 1, 1, 1, 1]]}

### Torch datasets

In [ ]:
class Dataset(torch.utils.data.Dataset):

  def __init__(self, encodings, labels=None):
    self.encodings = encodings
    self.labels = labels

  def __getitem__(self, idx):
    item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    if self.labels:
      item['labels'] = torch.tensor(self.labels[idx])
    return item

  def __len__(self):
    return len(self.encodings['input_ids'])

In [ ]:
len(X_train), len(X_test)

(800, 200)

In [ ]:
train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_test_tokenized, y_test)

In [ ]:
train_dataset

In [ ]:
train_dataset[2]

{'input_ids': tensor([  101,  1000,  2008,  2052,  2022,  3492,  5976,  2005,  2216, 11703,
         24938,  5491,  2000,  5254,  1037,  3185,  2008,  2001,  2580,  2044,
          1996,  2824,  2005,  2029,  2119,  1996,  3185,  1998,  1996,  5491,
          2079,  3141,  2055,  1012,  1996, 11703, 24938,  5491,  2020,  2025,
         23572,  2005,  1996,  3185,  1010,  4983,  2017,  2215,  2000,  6011,
          1037,  9714,  2006,  1996, 10752,  1006,  2200,  5976,  1007,  1012,
          2009,  2003,  6669,  9398,  2000,  2224,  1996,  5491,  2000, 14396,
          2000,  1996,  2168,  3439,  2824,  2000,  2582, 20410,  1996,  2054,
          3047,  1010,  2021,  1996,  2224,  1997,  3078,  3120,  1999,  2107,
          1037,  2001,  2003,  2025,  7812,  2005, 16948,  1012,  2144,  2009,
          2003,  2025,  7812,  1010,  2008,  2003,  2339,  1045,  2187,  2009,
          2182,  2006,  1996,  2831,  3931,  2065,  2619,  4122,  2000,  2470,
          2009,  2062,  2000,  2424,  7

In [ ]:
def model_metrics(p):
  print(type(p))

  pred, labels = p
  pred = np.argmax(pred, axis=1)


  accuracy = accuracy_score(y_true = labels, y_pred = pred)
  recall = recall_score(y_true = labels, y_pred = pred)
  precision = precision_score(y_true = labels, y_pred = pred)
  f1 = f1_score(y_true = labels, y_pred = pred)

  return {'accuracy': accuracy, "precision": precision, "recall": recall, "f1": f1}


#### Trainer

In [ ]:
!pip install transformers[torch]
!pip install accelerate -U

In [ ]:
!pip install accelerate>=0.21.0

In [ ]:
pip install accelerate

In [ ]:
# Define Trainer
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    output_dir = 'Bert-fine-class-twitter-comment',
    num_train_epochs = 2,
    per_device_train_batch_size = 8,


)

trainer = Trainer(
    model = model,
    args = args,
    train_dataset = train_dataset,
    eval_dataset = val_dataset,
    compute_metrics = model_metrics
)

In [ ]:
args = args.set_push_to_hub('amaithi/Bert-fintuned-classification-twitter-comments')
args

TrainingArguments(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
batch_eval_metrics=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=False,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_do_concat_batches=True,
eval_steps=None,
eval_strategy=no,
evaluation_strategy=None,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_c

In [ ]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=200, training_loss=0.0002576863579452038, metrics={'train_runtime': 148.4013, 'train_samples_per_second': 10.782, 'train_steps_per_second': 1.348, 'total_flos': 420977688576000.0, 'train_loss': 0.0002576863579452038, 'epoch': 2.0})

In [ ]:
trainer.evaluate()

<class 'transformers.trainer_utils.EvalPrediction'>


{'eval_loss': 0.6457701325416565,
 'eval_accuracy': 0.945,
 'eval_precision': 0.75,
 'eval_recall': 0.7142857142857143,
 'eval_f1': 0.7317073170731706,
 'eval_runtime': 5.5862,
 'eval_samples_per_second': 35.803,
 'eval_steps_per_second': 4.475,
 'epoch': 2.0}

In [ ]:
text = "that was good point"
# text = " Go to hell"

inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt").to('cuda')

outputs = model(**inputs)

# print(outputs)

predictions = torch.nn.functional.softmax(outputs.logits, dim=1)

# print(predictions)

pred = predictions.cpu().detach().numpy()

# pred = np.argmax(pred, axis=1)

pred

array([[9.9998963e-01, 1.0422485e-05]], dtype=float32)

In [ ]:
trainer.save_model('Bert-fintuned-classification-twitter-comments')

testing

In [ ]:
model2 = BertForSequenceClassification.from_pretrained('/content/Bert-fine-class-twitter-comment')

In [ ]:
model2 = model2.to('cuda')

In [ ]:
# text = "that was good point"
text = " Go to hell"

inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt").to('cuda')

outputs = model2(**inputs)

# print(outputs)

predictions = torch.nn.functional.softmax(outputs.logits, dim=1)

# print(predictions)

pred = predictions.cpu().detach().numpy()

pred = np.argmax(pred, axis=1)

pred

array([1])

### Push the model to HF

In [ ]:
pip install huggingface_hub

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
!huggingface-cli repo create Bert-fintuned-classification-twitter-comments

git version 2.34.1
git-lfs/3.0.2 (GitHub; linux amd64; go 1.18.1)

You are about to create amaithi/Bert-fintuned-classification-twitter-comments
Proceed? [Y/n] Y

Your repo now lives at:
  https://huggingface.co/amaithi/Bert-fintuned-classification-twitter-comments

You can clone it locally with the command below, and commit/push as usual.

  git clone https://huggingface.co/amaithi/Bert-fintuned-classification-twitter-comments



In [ ]:
# trainer = trainer.args.set(push_to_hub_model_id='amaithi/Bert-fintuned-classification-twitter-comments')

AttributeError: 'str' object has no attribute 'args'

In [ ]:
model2.push_to_hub('Bert-fintuned-classification-twitter-comments')

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/amaithi/Bert-fintuned-classification-twitter-comments/commit/54e4dbbeabf0f8451784e93624bbfa9f136db539', commit_message='Upload BertForSequenceClassification', commit_description='', oid='54e4dbbeabf0f8451784e93624bbfa9f136db539', pr_url=None, pr_revision=None, pr_num=None)